In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import seaborn as sns


# from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from scipy.stats import probplot
import gc
import matplotlib
# from pyod.models.ecod import ECOD
# import ppscore as pps

# # Build the Regression / Regressor models
# from xgboost import XGBClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
# from imblearn.over_sampling import BorderlineSMOTE
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.svm import SVC

# # import os
# import datetime
# from datetime import datetime as datet
# from scipy.stats import ttest_ind
import warnings
# from pickle import dump, load
# from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures, DropDuplicateFeatures

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mp.style.use('fivethirtyeight')

In [4]:
train = pd.read_csv("train.csv")
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [6]:
test = pd.read_csv("test.csv")
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [26]:
def data_quality(data : pd.core.frame.DataFrame, target_vars : list[str], unique_val_threshold : int = 20, num_to_cat_threshold : int = 20):
    cat_cols = []
    num_cols = []
    cat_cols_num_val = []
    date_time_cols = []
    data_info = pd.DataFrame(columns=['Column Name', 'Num Unique Values', 'DType', "Null Values", "Skew", "Kurtosis"])
    ctr = 1

    print("Shape:- ", data.shape[0], " rows & ", data.shape[1], " cols")
    print("\n--------------------------------------------------------\n")
    # print(data.isna().sum())
    # print("\n--------------------------------------------------------\n")

    for col in data.columns:
        # print(col, " - ", data[col].nunique(), " - ", data[col].dtypes, "\n")
        if data[col].dtypes == "int64" or data[col].dtypes == "float64":
            skew, kurtosis = data[col].skew(), data[col].kurtosis()
        else:
            skew, kurtosis = 0, 0

        data_info.loc[ctr] = [col, data[col].nunique(), data[col].dtypes, data[col].isna().sum(), skew, kurtosis]
        ctr += 1

        if data[col].nunique() <= unique_val_threshold:
            print(col)
            print(data[col].unique())
            print("----------------------------------------------------------------------------")
            print()

        if data[col].dtypes == "datetime64" or "date" in col.lower() or "time" in col.lower():
            date_time_cols.append(col)
        elif data[col].dtypes == object:
            cat_cols.append(col)
        elif data[col].dtypes == "int64" or data[col].dtypes == "float64":
            if data[col].nunique() <= num_to_cat_threshold:
                cat_cols_num_val.append(col)
            else:
                num_cols.append(col)   

    dups = data.drop(target_vars, axis = 1).duplicated().sum()
    print("Total number of Duplicates:- ", dups)
    print("\n--------------------------------------------------------\n")

    print("Categorical Columns:-\n", cat_cols, "\n")
    print("Numerical Columns:- \n", num_cols, "\n")
    print("Categorical Columns Having Numerical Values:- \n", cat_cols_num_val, "\n")
    print("Date Time Columns:-\n", date_time_cols, "\n")
    print("\n--------------------------------------------------------\n")

    return data_info, cat_cols, num_cols, cat_cols_num_val, date_time_cols

In [9]:
def func(p1 : int = 0, p2 : int = 2):
    return p1 + p2

In [28]:
train_info, cat_cols, num_cols, cat_cols_num_val, date_time_cols = data_quality(train, ['casual', 'registered', 'count'])

Shape:-  10886  rows &  12  cols

--------------------------------------------------------

season
[1 2 3 4]
----------------------------------------------------------------------------

holiday
[0 1]
----------------------------------------------------------------------------

workingday
[0 1]
----------------------------------------------------------------------------

weather
[1 2 3 4]
----------------------------------------------------------------------------

Total number of Duplicates:-  0

--------------------------------------------------------

Categorical Columns:-
 [] 

Numerical Columns:- 
 ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'] 

Categorical Columns Having Numerical Values:- 
 ['season', 'holiday', 'workingday', 'weather'] 

Date Time Columns:-
 ['datetime'] 


--------------------------------------------------------



In [30]:
train_info

,Column Name,Num Unique Values,DType,Null Values,Skew,Kurtosis
1,datetime,10886,object,0,0.000000,0.000000
2,season,4,int64,0,-0.007076,-1.355661
3,holiday,2,int64,0,5.660517,30.046975
4,workingday,2,int64,0,-0.776163,-1.397828
5,weather,4,int64,0,1.243484,0.395533
6,temp,49,float64,0,0.003691,-0.914530
7,atemp,60,float64,0,-0.102560,-0.850076
8,humidity,89,int64,0,-0.086335,-0.759818
9,windspeed,28,float64,0,0.588767,0.630133
10,casual,309,int64,0,2.495748,7.551629
